### Generate final results tables

This notebook uses the csv files with results generated by results_table.py and
aggregates and transforms them into final results tables for different experiments
that are then used to report results in the paper.

In [2]:
# import statements
import sys
sys.path.insert(0, "/home/atimans/Desktop/project_1/conformalbb")

from pathlib import Path
import pandas as pd

from evaluation import results_table
from util import util, io_file

#### Generate AP tables for predictive performance evaluation

In [ ]:
datasets = ["coco_val", "cityscapes", "bdd100k_train"]
files = ["std_conf_x101fpn_std_rank"]
paths = []

for d in datasets:
    for f in files:
        paths.append(
            f"output/{d}/{f}/{f}_ap_table.csv"
        )

In [ ]:
paths

['output/coco_val/std_conf_x101fpn_std_rank/std_conf_x101fpn_std_rank_ap_table.csv',
 'output/cityscapes/std_conf_x101fpn_std_rank/std_conf_x101fpn_std_rank_ap_table.csv',
 'output/bdd100k_train/std_conf_x101fpn_std_rank/std_conf_x101fpn_std_rank_ap_table.csv']

In [ ]:
rows = 3 # mean class (selected)
cols = ["AP@IOU=.50:.05:.95", "AP@IOU=.75", "AP@IOU=.50", "AP_small", "AP_medium", "AP_large"]
res_name = "res_ap_x101fpn"

In [ ]:
df_all = pd.DataFrame()

with open(f'results/{res_name}_latex.txt', 'w') as f:
    for path in paths:
        df = pd.read_csv(path)
        row_selected = df.loc[rows, cols]
        row_selected.fillna("--", inplace=True)
        df_all = df_all.append(row_selected, ignore_index=True)
        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)
df_all.to_csv(f'results/{res_name}_table.csv', index=False)

/tmp/ipykernel_424266/635425318.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(row_selected, ignore_index=True)
/tmp/ipykernel_424266/635425318.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(row_selected, ignore_index=True)
/tmp/ipykernel_424266/635425318.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(row_selected, ignore_index=True)


#### Label sets and box intervals produced by our conformal bounding box methods with our proposed max-rank correction

 Produces both box set tables (box metrics after label set procedure) and label set tables (label set metrics)
 
 Label set names
- match: box metrics for calibration samples with class matching and correct quantile selection (original), no label set procedure
- plain: box metrics for calibration samples without class matching but with correct quantile selection, no label set procedure
- full: box metrics for calibration samples without class matching and for full label sets
- top: box metrics for calibration samples without class matching and for top label sets
- oracle: box metrics for calibration samples without class matching and for density level label sets 
- class: box metrics for calibration samples without class matching and for class label sets (per-class thresholding)

Renaming (in line with the paper):
- Match --> OracleMatch (not reported in the paper)
- Plain --> Oracle
- Oracle --> Naive
- Class --> ClassThr

In [131]:
# only for single dataset at a time
d = "bdd100k_train"

res_folder = "results/results_selected_rank"
Path(res_folder).mkdir(exist_ok=True, parents=True)
res_box = f"{d}_res_rank_box_set" 
res_label = f"{d}_res_rank_label"

box_rows = 4 # mean class (selected)
box_cols = ["nr calib", "mpiw", "box stretch", "cov box", "cov box area S", "cov box area M", "cov box area L"]

label_rows = 4 # mean class (selected)
label_cols = ["nr calib", "null set frac", "mean set size", "cov set", "cov set area S", "cov set area M", "cov set area L"]

print(res_folder, "\n", res_box, "\n", res_label)

results/results_selected_rank 
 bdd100k_train_res_rank_box_set 
 bdd100k_train_res_rank_label


In [132]:
files = {
    "std_conf_x101fpn_std_rank": ["abs_res"], 
    "ens_conf_x101fpn_ens_rank": ["norm_res"], 
    "cqr_conf_x101fpn_cqr_rank": ["quant_res"]
         }
label = ["full", "top", "oracle", "class"]

conf_str = ["Box-Std", "Box-Ens", "Box-CQR"]
label_str = ["Full", "Top", "Naive", "ClassThr"]
label_plain = "Oracle"
label_match = "OracleMatch"

label_paths = []
box_paths = []
conf_col = []
label_col = []

for i, f in enumerate(files.keys()):
    for s in files[f]:
        
        box_paths.append(f"output/{d}/{f}/{f}_res_table_{s}.csv")
        conf_col.append(conf_str[i])
        label_col.append(label_match)
        
        # it doesnt matter for which label procedure here since before label sets results are identical
        box_paths.append(f"output/{d}/{f}_{label[-1]}/{f}_class_res_table_{s}.csv")
        conf_col.append(conf_str[i])
        label_col.append(label_plain)
        
        for j, l in enumerate(label):
            box_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_box_set_table_{s}.csv")
            label_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_label_table.csv")
            
            conf_col.append(conf_str[i])
            label_col.append(label_str[j])
            
for i in range(len(box_paths)):
    print(f"{conf_col[i]} | {label_col[i]} | {box_paths[i]}")

Box-Std | OracleMatch | output/bdd100k_train/std_conf_x101fpn_std_rank/std_conf_x101fpn_std_rank_res_table_abs_res.csv
Box-Std | Oracle | output/bdd100k_train/std_conf_x101fpn_std_rank_class/std_conf_x101fpn_std_rank_class_res_table_abs_res.csv
Box-Std | Full | output/bdd100k_train/std_conf_x101fpn_std_rank_full/std_conf_x101fpn_std_rank_full_box_set_table_abs_res.csv
Box-Std | Top | output/bdd100k_train/std_conf_x101fpn_std_rank_top/std_conf_x101fpn_std_rank_top_box_set_table_abs_res.csv
Box-Std | Naive | output/bdd100k_train/std_conf_x101fpn_std_rank_oracle/std_conf_x101fpn_std_rank_oracle_box_set_table_abs_res.csv
Box-Std | ClassThr | output/bdd100k_train/std_conf_x101fpn_std_rank_class/std_conf_x101fpn_std_rank_class_box_set_table_abs_res.csv
Box-Ens | OracleMatch | output/bdd100k_train/ens_conf_x101fpn_ens_rank/ens_conf_x101fpn_ens_rank_res_table_norm_res.csv
Box-Ens | Oracle | output/bdd100k_train/ens_conf_x101fpn_ens_rank_class/ens_conf_x101fpn_ens_rank_class_res_table_norm_res.

In [133]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_box}_latex.txt', 'w') as f:
    for i, path in enumerate(box_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)
        
        row_selected = df.loc[box_rows, box_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col[i], label_col[i]], index=["conf", "label"]), row_selected.loc[:]])

        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        # Convert the row to a string with '&' separating the values
        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_box}_table.csv', index=False, float_format=custom_format)

In [134]:
rmv_idx = [i for i, v in enumerate(label_col) if v == label_plain or v == label_match]
conf_col2 = [v for i, v in enumerate(conf_col) if i not in rmv_idx]
label_col2 = [v for i, v in enumerate(label_col) if i not in rmv_idx]

for i in range(len(label_paths)):
    print(f"{conf_col2[i]} / {label_col2[i]} / {label_paths[i]}")

Box-Std / Full / output/bdd100k_train/std_conf_x101fpn_std_rank_full/std_conf_x101fpn_std_rank_full_label_table.csv
Box-Std / Top / output/bdd100k_train/std_conf_x101fpn_std_rank_top/std_conf_x101fpn_std_rank_top_label_table.csv
Box-Std / Naive / output/bdd100k_train/std_conf_x101fpn_std_rank_oracle/std_conf_x101fpn_std_rank_oracle_label_table.csv
Box-Std / ClassThr / output/bdd100k_train/std_conf_x101fpn_std_rank_class/std_conf_x101fpn_std_rank_class_label_table.csv
Box-Ens / Full / output/bdd100k_train/ens_conf_x101fpn_ens_rank_full/ens_conf_x101fpn_ens_rank_full_label_table.csv
Box-Ens / Top / output/bdd100k_train/ens_conf_x101fpn_ens_rank_top/ens_conf_x101fpn_ens_rank_top_label_table.csv
Box-Ens / Naive / output/bdd100k_train/ens_conf_x101fpn_ens_rank_oracle/ens_conf_x101fpn_ens_rank_oracle_label_table.csv
Box-Ens / ClassThr / output/bdd100k_train/ens_conf_x101fpn_ens_rank_class/ens_conf_x101fpn_ens_rank_class_label_table.csv
Box-CQR / Full / output/bdd100k_train/cqr_conf_x101fpn_c

In [135]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0, 80.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_label}_latex.txt', 'w') as f:
    for i, path in enumerate(label_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)

        row_selected = df.loc[label_rows, label_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col2[i], label_col2[i]], index=["conf", "label"]), row_selected.loc[:]])
        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_label}_table.csv', index=False, float_format=custom_format)


In [136]:
row_selected

conf               Box-CQR
label             ClassThr
nr calib             55844
null set frac          0.0
mean set size       3.8318
cov set             0.9907
cov set area S      0.9719
cov set area M       0.993
cov set area L      0.9898
dtype: object

In [137]:
' & '.join(row_selected.astype(str)) + ' \\\\\n'

'Box-CQR & ClassThr & 55844 & 0.0 & 3.8318 & 0.9907 & 0.9719 & 0.993 & 0.9898 \\\\\n'

#### Baselines: Label sets and box intervals produced by our conformal bounding box methods with a Bonferroni correction

In [138]:
# only for single dataset at a time
# d = "coco_val"

res_folder = "results/results_selected_bonf"
Path(res_folder).mkdir(exist_ok=True, parents=True)
res_box = f"{d}_res_bonf_box_set" 
res_label = f"{d}_res_bonf_label"

box_rows = 4 # mean class (selected)
box_cols = ["nr calib", "mpiw", "box stretch", "cov box", "cov box area S", "cov box area M", "cov box area L"]

label_rows = 4 # mean class (selected)
label_cols = ["nr calib", "null set frac", "mean set size", "cov set", "cov set area S", "cov set area M", "cov set area L"]

print(res_folder, "\n", res_box, "\n", res_label)

results/results_selected_bonf 
 bdd100k_train_res_bonf_box_set 
 bdd100k_train_res_bonf_label


In [139]:
files = {
    "std_conf_x101fpn_std_bonf": ["abs_res"], 
    "ens_conf_x101fpn_ens_bonf": ["norm_res"], 
    "cqr_conf_x101fpn_cqr_bonf": ["quant_res"]
         }
label = ["full", "top", "oracle", "class"]

conf_str = ["Box-Std", "Box-Ens", "Box-CQR"]
label_str = ["Full", "Top", "Naive", "ClassThr"]
label_plain = "Oracle"
label_match = "OracleMatch"

label_paths = []
box_paths = []
conf_col = []
label_col = []

for i, f in enumerate(files.keys()):
    for s in files[f]:
        
        box_paths.append(f"output/{d}/{f}/{f}_res_table_{s}.csv")
        conf_col.append(conf_str[i])
        label_col.append(label_match)
        
        # it doesnt matter for which label procedure here since before label sets results are identical
        box_paths.append(f"output/{d}/{f}_{label[-1]}/{f}_class_res_table_{s}.csv")
        conf_col.append(conf_str[i])
        label_col.append(label_plain)
        
        for j, l in enumerate(label):
            box_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_box_set_table_{s}.csv")
            label_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_label_table.csv")
            
            conf_col.append(conf_str[i])
            label_col.append(label_str[j])
            
for i in range(len(box_paths)):
    print(f"{conf_col[i]} | {label_col[i]} | {box_paths[i]}")

Box-Std | OracleMatch | output/bdd100k_train/std_conf_x101fpn_std_bonf/std_conf_x101fpn_std_bonf_res_table_abs_res.csv
Box-Std | Oracle | output/bdd100k_train/std_conf_x101fpn_std_bonf_class/std_conf_x101fpn_std_bonf_class_res_table_abs_res.csv
Box-Std | Full | output/bdd100k_train/std_conf_x101fpn_std_bonf_full/std_conf_x101fpn_std_bonf_full_box_set_table_abs_res.csv
Box-Std | Top | output/bdd100k_train/std_conf_x101fpn_std_bonf_top/std_conf_x101fpn_std_bonf_top_box_set_table_abs_res.csv
Box-Std | Naive | output/bdd100k_train/std_conf_x101fpn_std_bonf_oracle/std_conf_x101fpn_std_bonf_oracle_box_set_table_abs_res.csv
Box-Std | ClassThr | output/bdd100k_train/std_conf_x101fpn_std_bonf_class/std_conf_x101fpn_std_bonf_class_box_set_table_abs_res.csv
Box-Ens | OracleMatch | output/bdd100k_train/ens_conf_x101fpn_ens_bonf/ens_conf_x101fpn_ens_bonf_res_table_norm_res.csv
Box-Ens | Oracle | output/bdd100k_train/ens_conf_x101fpn_ens_bonf_class/ens_conf_x101fpn_ens_bonf_class_res_table_norm_res.

In [140]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_box}_latex.txt', 'w') as f:
    for i, path in enumerate(box_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)
        
        row_selected = df.loc[box_rows, box_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col[i], label_col[i]], index=["conf", "label"]), row_selected.loc[:]])

        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        # Convert the row to a string with '&' separating the values
        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_box}_table.csv', index=False, float_format=custom_format)

In [141]:
rmv_idx = [i for i, v in enumerate(label_col) if v == label_plain or v == label_match]
conf_col2 = [v for i, v in enumerate(conf_col) if i not in rmv_idx]
label_col2 = [v for i, v in enumerate(label_col) if i not in rmv_idx]

for i in range(len(label_paths)):
    print(f"{conf_col2[i]} / {label_col2[i]} / {label_paths[i]}")

Box-Std / Full / output/bdd100k_train/std_conf_x101fpn_std_bonf_full/std_conf_x101fpn_std_bonf_full_label_table.csv
Box-Std / Top / output/bdd100k_train/std_conf_x101fpn_std_bonf_top/std_conf_x101fpn_std_bonf_top_label_table.csv
Box-Std / Naive / output/bdd100k_train/std_conf_x101fpn_std_bonf_oracle/std_conf_x101fpn_std_bonf_oracle_label_table.csv
Box-Std / ClassThr / output/bdd100k_train/std_conf_x101fpn_std_bonf_class/std_conf_x101fpn_std_bonf_class_label_table.csv
Box-Ens / Full / output/bdd100k_train/ens_conf_x101fpn_ens_bonf_full/ens_conf_x101fpn_ens_bonf_full_label_table.csv
Box-Ens / Top / output/bdd100k_train/ens_conf_x101fpn_ens_bonf_top/ens_conf_x101fpn_ens_bonf_top_label_table.csv
Box-Ens / Naive / output/bdd100k_train/ens_conf_x101fpn_ens_bonf_oracle/ens_conf_x101fpn_ens_bonf_oracle_label_table.csv
Box-Ens / ClassThr / output/bdd100k_train/ens_conf_x101fpn_ens_bonf_class/ens_conf_x101fpn_ens_bonf_class_label_table.csv
Box-CQR / Full / output/bdd100k_train/cqr_conf_x101fpn_c

In [142]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0, 80.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_label}_latex.txt', 'w') as f:
    for i, path in enumerate(label_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)

        row_selected = df.loc[label_rows, label_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col2[i], label_col2[i]], index=["conf", "label"]), row_selected.loc[:]])
        # df_all = df_all.append(row_selected, ignore_index=True)
        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_label}_table.csv', index=False, float_format=custom_format)


#### Baselines: Label sets and box intervals produced by conformal bounding box baselines

In [143]:
# only for single dataset at a time
d = "coco_val"

res_folder = "results/results_selected_base"
Path(res_folder).mkdir(exist_ok=True, parents=True)
res_box = f"{d}_res_base_box_set" 
res_label = f"{d}_res_base_label"

box_rows = 4 # mean class (selected)
box_cols = ["nr calib", "mpiw", "box stretch", "cov box", "cov box area S", "cov box area M", "cov box area L"]

label_rows = 4 # mean class (selected)
label_cols = ["nr calib", "null set frac", "mean set size", "cov set", "cov set area S", "cov set area M", "cov set area L"]

print(res_folder, "\n", res_box, "\n", res_label)

results/results_selected_base 
 bdd100k_train_res_base_box_set 
 bdd100k_train_res_base_label


In [144]:
files = {
    "base_conf_x101fpn_base_bonf": ["one_sided_res", "one_sided_mult_res"],
    "base_conf_x101fpn_base_max": ["one_sided_res", "one_sided_mult_res"], 
    "base_conf_x101fpn_base_rank": ["one_sided_res", "one_sided_mult_res"],
    "ens_conf_x101fpn_base_ens": ["norm_res"]
         }
label = ["full", "top", "oracle", "class"]

conf_str = [
    ["add + Bonf", "mult + Bonf"], 
    ["add + max", "mult + max"], 
    ["add + max-rank", "mult + max-rank"], 
    ["Box-Ens + mult-rank"]]
label_str = ["Full", "Top", "Naive", "ClassThr"]
label_plain = "Oracle"
label_match = "OracleMatch"

label_paths = []
box_paths = []
conf_col = []
label_col = []

for i, f in enumerate(files.keys()):
    for ii, s in enumerate(files[f]):
        
        box_paths.append(f"output/{d}/{f}/{f}_res_table_{s}.csv")
        conf_col.append(conf_str[i][ii])
        label_col.append(label_match)
        
        # it doesnt matter for which label procedure here since before label sets results are identical
        box_paths.append(f"output/{d}/{f}_{label[-1]}/{f}_class_res_table_{s}.csv")
        conf_col.append(conf_str[i][ii])
        label_col.append(label_plain)
        
        for j, l in enumerate(label):
            box_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_box_set_table_{s}.csv")
            label_paths.append(f"output/{d}/{f}_{l}/{f}_{l}_label_table.csv")
            
            conf_col.append(conf_str[i][ii])
            label_col.append(label_str[j])
            
for i in range(len(box_paths)):
    print(f"{conf_col[i]} | {label_col[i]} | {box_paths[i]}")

add + Bonf | OracleMatch | output/bdd100k_train/base_conf_x101fpn_base_bonf/base_conf_x101fpn_base_bonf_res_table_one_sided_res.csv
add + Bonf | Oracle | output/bdd100k_train/base_conf_x101fpn_base_bonf_class/base_conf_x101fpn_base_bonf_class_res_table_one_sided_res.csv
add + Bonf | Full | output/bdd100k_train/base_conf_x101fpn_base_bonf_full/base_conf_x101fpn_base_bonf_full_box_set_table_one_sided_res.csv
add + Bonf | Top | output/bdd100k_train/base_conf_x101fpn_base_bonf_top/base_conf_x101fpn_base_bonf_top_box_set_table_one_sided_res.csv
add + Bonf | Naive | output/bdd100k_train/base_conf_x101fpn_base_bonf_oracle/base_conf_x101fpn_base_bonf_oracle_box_set_table_one_sided_res.csv
add + Bonf | ClassThr | output/bdd100k_train/base_conf_x101fpn_base_bonf_class/base_conf_x101fpn_base_bonf_class_box_set_table_one_sided_res.csv
mult + Bonf | OracleMatch | output/bdd100k_train/base_conf_x101fpn_base_bonf/base_conf_x101fpn_base_bonf_res_table_one_sided_mult_res.csv
mult + Bonf | Oracle | outp

In [145]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_box}_latex.txt', 'w') as f:
    for i, path in enumerate(box_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)
        
        row_selected = df.loc[box_rows, box_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col[i], label_col[i]], index=["conf", "label"]), row_selected.loc[:]])

        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        # Convert the row to a string with '&' separating the values
        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_box}_table.csv', index=False, float_format=custom_format)

In [146]:
rmv_idx = [i for i, v in enumerate(label_col) if v == label_plain or v == label_match]
conf_col2 = [v for i, v in enumerate(conf_col) if i not in rmv_idx]
label_col2 = [v for i, v in enumerate(label_col) if i not in rmv_idx]

for i in range(len(label_paths)):
    print(f"{conf_col2[i]} / {label_col2[i]} / {label_paths[i]}")

add + Bonf / Full / output/bdd100k_train/base_conf_x101fpn_base_bonf_full/base_conf_x101fpn_base_bonf_full_label_table.csv
add + Bonf / Top / output/bdd100k_train/base_conf_x101fpn_base_bonf_top/base_conf_x101fpn_base_bonf_top_label_table.csv
add + Bonf / Naive / output/bdd100k_train/base_conf_x101fpn_base_bonf_oracle/base_conf_x101fpn_base_bonf_oracle_label_table.csv
add + Bonf / ClassThr / output/bdd100k_train/base_conf_x101fpn_base_bonf_class/base_conf_x101fpn_base_bonf_class_label_table.csv
mult + Bonf / Full / output/bdd100k_train/base_conf_x101fpn_base_bonf_full/base_conf_x101fpn_base_bonf_full_label_table.csv
mult + Bonf / Top / output/bdd100k_train/base_conf_x101fpn_base_bonf_top/base_conf_x101fpn_base_bonf_top_label_table.csv
mult + Bonf / Naive / output/bdd100k_train/base_conf_x101fpn_base_bonf_oracle/base_conf_x101fpn_base_bonf_oracle_label_table.csv
mult + Bonf / ClassThr / output/bdd100k_train/base_conf_x101fpn_base_bonf_class/base_conf_x101fpn_base_bonf_class_label_table.

In [147]:
df_all = pd.DataFrame()

# Custom float format function
def custom_format(x):
    if x in [0.0, 1.0, 80.0]:
        return x
    else:
        return '{:.4f}'.format(x)

with open(f'{res_folder}/{res_label}_latex.txt', 'w') as f:
    for i, path in enumerate(label_paths):
        df = pd.read_csv(path)
        
        df['nr calib'] = df['nr calib'].round(0).astype(int)

        row_selected = df.loc[label_rows, label_cols]
        row_selected.fillna("--", inplace=True)
        
        # extend by conf and label str
        row_selected = pd.concat([pd.Series([conf_col2[i], label_col2[i]], index=["conf", "label"]), row_selected.loc[:]])
        df_all = pd.concat([df_all, row_selected.to_frame().T], ignore_index=True)

        row_string = ' & '.join(row_selected.astype(str)) + ' \\\\\n'
        f.write(row_string)

df_all.to_csv(f'{res_folder}/{res_label}_table.csv', index=False, float_format=custom_format)


#### Baselines: DeepEns, GaussianYOLO

In [ ]:
dataset = "coco_val"
filedir = f"../../../../media/atimans/hdd/output_90_99/{dataset}"

classes = list(util.get_selected_coco_classes().values())
i, j = 0, 4 # score indices

In [28]:
deep_ens = io_file.load_tensor("deep_ens_x101fpn_base_deep_ens_control", f"{filedir}/deep_ens_x101fpn_base_deep_ens")
deep_ens_onesided = io_file.load_tensor("deep_ens_x101fpn_base_deep_ens_onesided_control", f"{filedir}/deep_ens_x101fpn_base_deep_ens_onesided")
gaussian_yolo = io_file.load_tensor("gaussian_yolo_x101fpn_base_gaussian_yolo_control", f"{filedir}/gaussian_yolo_x101fpn_base_gaussian_yolo")
gaussian_yolo_onesided = io_file.load_tensor("gaussian_yolo_x101fpn_base_gaussian_yolo_onesided_control", f"{filedir}/gaussian_yolo_x101fpn_base_gaussian_yolo_onesided")
base_add_max = io_file.load_tensor("base_conf_x101fpn_base_max_class_control", f"{filedir}/base_conf_x101fpn_base_max_class")
box_std = io_file.load_tensor("std_conf_x101fpn_std_rank_class_control", f"{filedir}/std_conf_x101fpn_std_rank_class")
box_std_onesided = io_file.load_tensor("base_conf_x101fpn_base_rank_class_control", f"{filedir}/base_conf_x101fpn_base_rank_class")

Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/deep_ens_x101fpn_base_deep_ens/deep_ens_x101fpn_base_deep_ens_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/deep_ens_x101fpn_base_deep_ens_onesided/deep_ens_x101fpn_base_deep_ens_onesided_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/gaussian_yolo_x101fpn_base_gaussian_yolo/gaussian_yolo_x101fpn_base_gaussian_yolo_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/gaussian_yolo_x101fpn_base_gaussian_yolo_onesided/gaussian_yolo_x101fpn_base_gaussian_yolo_onesided_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/base_conf_x101fpn_base_max_class/base_conf_x101fpn_base_max_class_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output_90_99/coco_val/std_conf_x101fpn_std_rank_class/std_conf_x101fpn_std_rank_class_control.pt.
Loaded torch

In [29]:
results_table._idx_metrics

{'nr_calib_samp': 0,
 'quant': 1,
 'mpiw': 2,
 'box_stretch': 3,
 'cov_coord': 4,
 'cov_box': 5,
 'cov_area': [6, 9],
 'cov_iou': [9, 12]}

In [31]:
deep_ens.shape, deep_ens_onesided.shape, gaussian_yolo.shape, gaussian_yolo_onesided.shape, base_add_max.shape, box_std.shape, box_std_onesided.shape

(torch.Size([100, 80, 4, 12]),
 torch.Size([100, 80, 4, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 4, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]))

In [35]:
# sanity check
print("MPIW | Box cov")
print(deep_ens[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", deep_ens[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(deep_ens_onesided[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", deep_ens_onesided[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(gaussian_yolo[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", gaussian_yolo[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(gaussian_yolo_onesided[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", gaussian_yolo_onesided[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(base_add_max[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", base_add_max[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(box_std[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", box_std[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))
print(box_std_onesided[:, classes, i:j].mean(dim=(1,2))[:, 2].mean(dim=0), " | ", box_std_onesided[:, classes, i:j].mean(dim=(1,2))[:, 5].mean(dim=0))

MPIW | Box cov
tensor(12.3084)  |  tensor(0.2092)
tensor(74.1486)  |  tensor(0.4880)
tensor(6.9992)  |  tensor(0.0784)
tensor(87.0675)  |  tensor(0.3519)
tensor(87.6190)  |  tensor(0.9085)
tensor(55.4745)  |  tensor(0.8836)
tensor(85.4203)  |  tensor(0.8850)


In [36]:
cov_deep_ens = deep_ens[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_deep_ens = deep_ens[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_deep_ens_onesided = deep_ens_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_deep_ens_onesided = deep_ens_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_gaussian_yolo = gaussian_yolo[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_gaussian_yolo = gaussian_yolo[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_gaussian_yolo_onesided = gaussian_yolo_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_gaussian_yolo_onesided = gaussian_yolo_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_base_add_max = base_add_max[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_base_add_max = base_add_max[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_box_std = box_std[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_box_std = box_std[:, classes, i:j].nanmean(dim=(1,2))[:, 2]
cov_box_std_onesided = box_std_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 5]
mpiw_box_std_onesided = box_std_onesided[:, classes, i:j].nanmean(dim=(1,2))[:, 2]

In [39]:
print("Twosided")
print("Method | MPIW | Box cov")
print(f"DeepEns | {mpiw_deep_ens.mean():.2f} +/- {mpiw_deep_ens.std():.2f} | {cov_deep_ens.mean():.2f} +/- {cov_deep_ens.std():.2f}")
print(f"GaussianYolo | {mpiw_gaussian_yolo.mean():.2f} +/- {mpiw_gaussian_yolo.std():.2f} | {cov_gaussian_yolo.mean():.2f} +/- {cov_gaussian_yolo.std():.2f}")
print(f"BoxStd | {mpiw_box_std.mean():.2f} +/- {mpiw_box_std.std():.2f} | {cov_box_std.mean():.2f} +/- {cov_box_std.std():.2f}")

Twosided
Method | MPIW | Box cov
DeepEns | 12.31 +/- 0.47 | 0.21 +/- 0.01
GaussianYolo | 7.00 +/- 0.14 | 0.08 +/- 0.01
BoxStd | 55.47 +/- 2.97 | 0.88 +/- 0.02


In [40]:
print("Onesided")
print("Method | MPIW | Box cov")
print(f"DeepEnsOnesided | {mpiw_deep_ens_onesided.mean():.2f} +/- {mpiw_deep_ens_onesided.std():.2f} | {cov_deep_ens_onesided.mean():.2f} +/- {cov_deep_ens_onesided.std():.2f}")
print(f"GaussianYoloOnesided | {mpiw_gaussian_yolo_onesided.mean():.2f} +/- {mpiw_gaussian_yolo_onesided.std():.2f} | {cov_gaussian_yolo_onesided.mean():.2f} +/- {cov_gaussian_yolo_onesided.std():.2f}")
print(f"BaseAddMax | {mpiw_base_add_max.mean():.2f} +/- {mpiw_base_add_max.std():.2f} | {cov_base_add_max.mean():.2f} +/- {cov_base_add_max.std():.2f}")
print(f"BoxStdOnesided | {mpiw_box_std_onesided.mean():.2f} +/- {mpiw_box_std_onesided.std():.2f} | {cov_box_std_onesided.mean():.2f} +/- {cov_box_std_onesided.std():.2f}")

Onesided
Method | MPIW | Box cov
DeepEnsOnesided | 74.15 +/- 2.01 | 0.49 +/- 0.01
GaussianYoloOnesided | 87.07 +/- 4.25 | 0.35 +/- 0.01
BaseAddMax | 87.62 +/- 1.79 | 0.91 +/- 0.01
BoxStdOnesided | 85.42 +/- 1.99 | 0.88 +/- 0.02


#### Baselines: YOLOv3, DETR, Sparse R-CNN

In [4]:
dataset = "coco_val"
filedir = f"../../../../media/atimans/hdd/output/{dataset}"

classes = list(util.get_selected_coco_classes().values())
# i, j = 0, 4 # score indices

In [6]:
yolo = io_file.load_tensor("std_conf_yolov3_base_std_rank_control", f"{filedir}/std_conf_yolov3_base_std_rank")
yolo_base = io_file.load_tensor("std_conf_yolov3_base_std_max_control", f"{filedir}/std_conf_yolov3_base_std_max")

detr = io_file.load_tensor("std_conf_detr_std_rank_control", f"{filedir}/std_conf_detr_std_rank")
detr_base = io_file.load_tensor("std_conf_detr_base_std_max_control", f"{filedir}/std_conf_detr_base_std_max")

rcnn = io_file.load_tensor("std_conf_sparsercnn_base_std_rank_control", f"{filedir}/std_conf_sparsercnn_base_std_rank")
rcnn_base = io_file.load_tensor("std_conf_sparsercnn_base_std_max_control", f"{filedir}/std_conf_sparsercnn_base_std_max")


yolo.shape, yolo_base.shape, detr.shape, detr_base.shape, rcnn.shape, rcnn_base.shape

Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_yolov3_base_std_rank/std_conf_yolov3_base_std_rank_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_yolov3_base_std_max/std_conf_yolov3_base_std_max_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_detr_std_rank/std_conf_detr_std_rank_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_detr_base_std_max/std_conf_detr_base_std_max_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_sparsercnn_base_std_rank/std_conf_sparsercnn_base_std_rank_control.pt.
Loaded torch tensor from ../../../../media/atimans/hdd/output/coco_val/std_conf_sparsercnn_base_std_max/std_conf_sparsercnn_base_std_max_control.pt.


(torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]),
 torch.Size([100, 80, 8, 12]))

In [7]:
yolo_two = yolo[:, classes, 0:4, :]
yolo_one = yolo[:, classes, 4:, :]
yolo_base_one = yolo_base[:, classes, 4:, :]
detr_two = detr[:, classes, 0:4, :]
detr_one = detr[:, classes, 4:, :]
detr_base_one = detr_base[:, classes, 4:, :]
rcnn_two = rcnn[:, classes, 0:4, :]
rcnn_one = rcnn[:, classes, 4:, :]
rcnn_base_one = rcnn_base[:, classes, 4:, :]

yolo_two.shape, yolo_one.shape, yolo_base_one.shape, detr_two.shape, detr_one.shape, detr_base_one.shape, rcnn_two.shape, rcnn_one.shape, rcnn_base_one.shape

(torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]),
 torch.Size([100, 6, 4, 12]))

In [9]:
# sanity check
print("MPIW | Box cov")
print(yolo_two.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", yolo_two.mean(dim=(1,2))[:, 5].mean(dim=0))
print(yolo_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", yolo_one.mean(dim=(1,2))[:, 5].mean(dim=0))
print(yolo_base_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", yolo_base_one.mean(dim=(1,2))[:, 5].mean(dim=0))
print(detr_two.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", detr_two.mean(dim=(1,2))[:, 5].mean(dim=0))
print(detr_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", detr_one.mean(dim=(1,2))[:, 5].mean(dim=0))
print(detr_base_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", detr_base_one.mean(dim=(1,2))[:, 5].mean(dim=0))
print(rcnn_two.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", rcnn_two.mean(dim=(1,2))[:, 5].mean(dim=0))
print(rcnn_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", rcnn_one.mean(dim=(1,2))[:, 5].mean(dim=0))
print(rcnn_base_one.mean(dim=(1,2))[:, 2].mean(dim=0), " | ", rcnn_base_one.mean(dim=(1,2))[:, 5].mean(dim=0))

MPIW | Box cov
tensor(61.7268)  |  tensor(0.8772)
tensor(103.1217)  |  tensor(0.8775)
tensor(107.9278)  |  tensor(0.9158)
tensor(45.3426)  |  tensor(0.8837)
tensor(80.5734)  |  tensor(0.8834)
tensor(82.2095)  |  tensor(0.9042)
tensor(41.9183)  |  tensor(0.8877)
tensor(77.3255)  |  tensor(0.8885)
tensor(79.3474)  |  tensor(0.9057)


In [10]:
mpiw_yolo_two = yolo_two.nanmean(dim=(1,2))[:, 2]
cov_yolo_two = yolo_two.nanmean(dim=(1,2))[:, 5]
mpiw_yolo_one = yolo_one.nanmean(dim=(1,2))[:, 2]
cov_yolo_one = yolo_one.nanmean(dim=(1,2))[:, 5]
mpiw_yolo_base_one = yolo_base_one.nanmean(dim=(1,2))[:, 2]
cov_yolo_base_one = yolo_base_one.nanmean(dim=(1,2))[:, 5]

mpiw_detr_two = detr_two.nanmean(dim=(1,2))[:, 2]
cov_detr_two = detr_two.nanmean(dim=(1,2))[:, 5]
mpiw_detr_one = detr_one.nanmean(dim=(1,2))[:, 2]
cov_detr_one = detr_one.nanmean(dim=(1,2))[:, 5]
mpiw_detr_base_one = detr_base_one.nanmean(dim=(1,2))[:, 2]
cov_detr_base_one = detr_base_one.nanmean(dim=(1,2))[:, 5]

mpiw_rcnn_two = rcnn_two.nanmean(dim=(1,2))[:, 2]
cov_rcnn_two = rcnn_two.nanmean(dim=(1,2))[:, 5]
mpiw_rcnn_one = rcnn_one.nanmean(dim=(1,2))[:, 2]
cov_rcnn_one = rcnn_one.nanmean(dim=(1,2))[:, 5]
mpiw_rcnn_base_one = rcnn_base_one.nanmean(dim=(1,2))[:, 2]
cov_rcnn_base_one = rcnn_base_one.nanmean(dim=(1,2))[:, 5]


In [11]:
print("Twosided")
print("Method | MPIW | Box cov")
print(f"YOLOv3 | {mpiw_yolo_two.mean():.2f} +/- {mpiw_yolo_two.std():.2f} | {cov_yolo_two.mean():.2f} +/- {cov_yolo_two.std():.2f}")
print(f"DETR | {mpiw_detr_two.mean():.2f} +/- {mpiw_detr_two.std():.2f} | {cov_detr_two.mean():.2f} +/- {cov_detr_two.std():.2f}")
print(f"Sparse R-CNN | {mpiw_rcnn_two.mean():.2f} +/- {mpiw_rcnn_two.std():.2f} | {cov_rcnn_two.mean():.2f} +/- {cov_rcnn_two.std():.2f}")

Twosided
Method | MPIW | Box cov
YOLOv3 | 61.73 +/- 3.66 | 0.88 +/- 0.02
DETR | 45.34 +/- 3.33 | 0.88 +/- 0.02
Sparse R-CNN | 41.92 +/- 2.16 | 0.89 +/- 0.01


In [12]:
print("Onesided")
print("Method | MPIW | Box cov")
print(f"YOLOv3 Base | {mpiw_yolo_base_one.mean():.2f} +/- {mpiw_yolo_base_one.std():.2f} | {cov_yolo_base_one.mean():.2f} +/- {cov_yolo_base_one.std():.2f}")
print(f"YOLOv3 Ours | {mpiw_yolo_one.mean():.2f} +/- {mpiw_yolo_one.std():.2f} | {cov_yolo_one.mean():.2f} +/- {cov_yolo_one.std():.2f}")
print(f"DETR Base | {mpiw_detr_base_one.mean():.2f} +/- {mpiw_detr_base_one.std():.2f} | {cov_detr_base_one.mean():.2f} +/- {cov_detr_base_one.std():.2f}")
print(f"DETR Ours | {mpiw_detr_one.mean():.2f} +/- {mpiw_detr_one.std():.2f} | {cov_detr_one.mean():.2f} +/- {cov_detr_one.std():.2f}")
print(f"Sparse R-CNN Base | {mpiw_rcnn_base_one.mean():.2f} +/- {mpiw_rcnn_base_one.std():.2f} | {cov_rcnn_base_one.mean():.2f} +/- {cov_rcnn_base_one.std():.2f}")
print(f"Sparse R-CNN Ours | {mpiw_rcnn_one.mean():.2f} +/- {mpiw_rcnn_one.std():.2f} | {cov_rcnn_one.mean():.2f} +/- {cov_rcnn_one.std():.2f}")

Onesided
Method | MPIW | Box cov
YOLOv3 Base | 107.93 +/- 4.85 | 0.92 +/- 0.02
YOLOv3 Ours | 103.12 +/- 3.95 | 0.88 +/- 0.02
DETR Base | 82.21 +/- 1.64 | 0.90 +/- 0.01
DETR Ours | 80.57 +/- 1.78 | 0.88 +/- 0.01
Sparse R-CNN Base | 79.35 +/- 1.78 | 0.91 +/- 0.01
Sparse R-CNN Ours | 77.33 +/- 1.72 | 0.89 +/- 0.01
